In [1]:
# Packages
import torch, os
import torch.nn as nn
import torch.utils.data as utils
import torch.nn.functional as F
import numpy as np
import pandas as pd
import time
from torch.autograd import Variable
import math
import copy
     

In [2]:
%cd '/content/drive/MyDrive/TRAFFIC4CAST_TRBAI'

/content/drive/MyDrive/TRAFFIC4CAST_TRBAI


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!gdown 'https://drive.google.com/uc?id=1E_qqe7kfvfApM4hCOBMPoXhyEPyrUJkN'
!gdown 'https://drive.google.com/uc?id=1j-3-lHegY--FDHKZvz86HBtilV2dsP-i'
!gdown 'https://drive.google.com/uc?id=1AhAb3yEVxcHSaz0ADlyyYwNH2Gqmr-um'


Downloading...
From: https://drive.google.com/uc?id=1E_qqe7kfvfApM4hCOBMPoXhyEPyrUJkN
To: /content/drive/MyDrive/TRAFFIC4CAST_TRBAI/train.zip
100% 8.67M/8.67M [00:00<00:00, 97.3MB/s]
Downloading...
From: https://drive.google.com/uc?id=1j-3-lHegY--FDHKZvz86HBtilV2dsP-i
To: /content/drive/MyDrive/TRAFFIC4CAST_TRBAI/test.zip
100% 459k/459k [00:00<00:00, 106MB/s]
Downloading...
From: https://drive.google.com/uc?id=1AhAb3yEVxcHSaz0ADlyyYwNH2Gqmr-um
To: /content/drive/MyDrive/TRAFFIC4CAST_TRBAI/Geoshapefile.zip
100% 53.4k/53.4k [00:00<00:00, 69.7MB/s]


In [ ]:
!unzip 'train.zip' -d '.'
!unzip 'test.zip' -d '.'
!unzip 'Geoshapefile.zip' -d '.'

Archive:  train.zip
  inflating: ./tps_df.pkl            
  inflating: ./__MACOSX/._tps_df.pkl  
Archive:  test.zip
  inflating: ./test_data/tps_5.pkl   
  inflating: ./test_data/tps_2.pkl   
  inflating: ./test_data/tps_6.pkl   
  inflating: ./test_data/tps_4.pkl   
  inflating: ./test_data/tps_15.pkl  
  inflating: ./test_data/tps_7.pkl   
  inflating: ./test_data/tps_1.pkl   
  inflating: ./test_data/tps_12.pkl  
  inflating: ./test_data/tps_8.pkl   
  inflating: ./test_data/tps_3.pkl   
  inflating: ./test_data/tps_9.pkl   
  inflating: ./test_data/tps_13.pkl  
  inflating: ./test_data/tps_11.pkl  
  inflating: ./test_data/tps_10.pkl  
  inflating: ./test_data/tps_14.pkl  
Archive:  Geoshapefile.zip
  inflating: ./Geoshapefile/hackathon.cpg  
  inflating: ./Geoshapefile/hackathon.shx  
  inflating: ./Geoshapefile/hackathon.prj  
  inflating: ./Geoshapefile/hackathon.dbf  
  inflating: ./Geoshapefile/hackathon.shp  


In [3]:
# Data Preparation #define the function of reshape data to retunr reshaped_tps_df is ready, for each unique segment
def reshape_data(rawdata):
    reshaped_tps_df = pd.DataFrame()
    reshaped_tps_df['TIME'] = rawdata.time.unique()
    for seg in rawdata.segmentID.unique():
        column = rawdata[rawdata['segmentID'] == seg][['time','TrafficIndex_GP']].drop_duplicates(subset=['time'])
        column.columns = ['TIME', str(seg)]
        reshaped_tps_df = reshaped_tps_df.join(column.set_index('TIME'), on='TIME')

    return reshaped_tps_df        

    

In [4]:
# Data preparaiton , define the function of Laod_data, uses the reshaoed_tps_df as a matrix and prepare the time intevals, retunr output
def load_data(filepath, start_time, end_time, freq):
    rawdata = pd.read_pickle(filepath)
    matrix = reshape_data(rawdata)
    matrix['TIME'] = pd.to_datetime(matrix['TIME']).dt.strftime('%Y-%m-%d %H:%M:%S')

    dt_idx = pd.date_range(start=start_time, end=end_time, freq=freq)

    output = pd.DataFrame(dt_idx)
    output.columns = ['TIME']
    output['TIME'] = pd.to_datetime(output['TIME']).dt.strftime('%Y-%m-%d %H:%M:%S')
    output = output.set_index('TIME').join(matrix.set_index('TIME'))

    return output

In [5]:
#data loader fnction, takes the last out put ( prepared data)
def prepare_dataloader(matrix, n_col, seq_len=36, pred_len=12, BATCH_SIZE=32, device='cpu'):
    seg = matrix.columns.values
    time = matrix.index.values
    n_seg = len(seg)
    n_time = len(time)
    
    speedMatrix = matrix.to_numpy()
    
    data_set = []
    label_set = []

    for i in range(n_time - seq_len - pred_len):
        data = speedMatrix[i : i + seq_len]
        
        label_data = speedMatrix[i + seq_len: i + seq_len + pred_len, :n_col]
        
        if np.isnan(np.sum(data[:n_col])).any() | np.isnan(np.sum(label_data)):
            pass
        else:

            data_set.append(data)
            label_set.append(label_data)


  
#####data prepared>       


    data = np.array(data_set)
    label = np.array(label_set)



    train_ind = int(len(data)* 0.8)
    valid_ind = int(len(data) * 0.9)
    test_ind = int(len(data) * 1.0)


######extrating  Training Validation and Test data


    X_train = data[: train_ind]
    X_valid = data[train_ind : valid_ind]
    X_test = data[valid_ind : test_ind]
    Y_train = label[: train_ind]
    Y_valid = label[train_ind : valid_ind]
    Y_test = label[valid_ind : test_ind]


#Tensors
    X_train = torch.FloatTensor(X_train).to(device)
    X_valid = torch.FloatTensor(X_valid).to(device)
    X_test = torch.FloatTensor(X_test).to(device)
    Y_train = torch.FloatTensor(Y_train).to(device)
    Y_valid = torch.FloatTensor(Y_valid).to(device)
    Y_test = torch.FloatTensor(Y_test).to(device)

    train_dataset = utils.TensorDataset(X_train, Y_train)
    valid_dataset = utils.TensorDataset(X_valid, Y_valid)
    test_dataset = utils.TensorDataset(X_test, Y_test)

###Batch size already define in the function
    train_dataloader = utils.DataLoader(train_dataset, batch_size = BATCH_SIZE, shuffle=True, drop_last = True)
    valid_dataloader = utils.DataLoader(valid_dataset, batch_size = BATCH_SIZE, shuffle=True, drop_last = True)
    test_dataloader = utils.DataLoader(test_dataset, batch_size = BATCH_SIZE, shuffle=False, drop_last = False)

##Training Validation and Test data

    return train_dataloader, valid_dataloader, test_dataloader


    # Data prepration done!

In [6]:
# Encoder Architecture   include __init__ and forward function 


class Encoder(nn.Module):

    def __init__(self, encoder_layer, number_of_layers):
        super().__init__()
        assert isinstance(encoder_layer, EncoderLayer), f'Expected EncoderLayer got {type(encoder_layer)}.'
        self.encoder_layers = get_clones(encoder_layer, number_of_layers)
        self.norm = nn.LayerNorm(encoder_layer.model_dimension)

    def forward(self, src_embeddings_batch, src_mask):
        src_representations_batch = src_embeddings_batch
        for encoder_layer in self.encoder_layers:
            src_representations_batch = encoder_layer(src_representations_batch, src_mask)
        return self.norm(src_representations_batch)


In [7]:
# Encoder Layer architecture defined a Class like the encoder itself,
#src_representations_batch

class EncoderLayer(nn.Module):

    def __init__(self, model_dimension, dropout_probability, multi_headed_attention, pointwise_net):
        super().__init__()
        num_of_sublayers_encoder = 2
        self.sublayers = get_clones(SublayerLogic(model_dimension, dropout_probability), num_of_sublayers_encoder)

        self.multi_headed_attention = multi_headed_attention
        self.pointwise_net = pointwise_net

        self.model_dimension = model_dimension

    def forward(self, src_representations_batch, src_mask):
        encoder_self_attention = lambda srb: self.multi_headed_attention(query=srb, key=srb, value=srb, mask=src_mask)
        src_representations_batch = self.sublayers[0](src_representations_batch, encoder_self_attention)
        src_representations_batch = self.sublayers[1](src_representations_batch, self.pointwise_net)

        return src_representations_batch
        

In [8]:
# Decoder Architecture returns "trg_representations_batch"
class Decoder(nn.Module):

    def __init__(self, decoder_layer, number_of_layers):
        super().__init__()
        assert isinstance(decoder_layer, DecoderLayer), f'Expected DecoderLayer got {type(decoder_layer)}.'

        self.decoder_layers = get_clones(decoder_layer, number_of_layers)
        self.norm = nn.LayerNorm(decoder_layer.model_dimension)

    def forward(self, trg_embeddings_batch, src_representations_batch, trg_mask, src_mask):
        trg_representations_batch = trg_embeddings_batch
        for decoder_layer in self.decoder_layers:
            trg_representations_batch = decoder_layer(trg_representations_batch, src_representations_batch, trg_mask, src_mask)
        return self.norm(trg_representations_batch)

class DecoderLayer(nn.Module):

    def __init__(self, model_dimension, dropout_probability, multi_headed_attention, pointwise_net):
        super().__init__()
        num_of_sublayers_decoder = 3
        self.sublayers = get_clones(SublayerLogic(model_dimension, dropout_probability), num_of_sublayers_decoder)

        self.trg_multi_headed_attention = copy.deepcopy(multi_headed_attention)
        self.src_multi_headed_attention = copy.deepcopy(multi_headed_attention)
        self.pointwise_net = pointwise_net

        self.model_dimension = model_dimension

    def forward(self, trg_representations_batch, src_representations_batch, trg_mask, src_mask):
        srb = src_representations_batch
        decoder_trg_self_attention = lambda trb: self.trg_multi_headed_attention(query=trb, key=trb, value=trb, mask=trg_mask)
        decoder_src_attention = lambda trb: self.src_multi_headed_attention(query=trb, key=srb, value=srb, mask=src_mask)

        trg_representations_batch = self.sublayers[0](trg_representations_batch, decoder_trg_self_attention)
        trg_representations_batch = self.sublayers[1](trg_representations_batch, decoder_src_attention)
        trg_representations_batch = self.sublayers[2](trg_representations_batch, self.pointwise_net)

        return trg_representations_batch

In [9]:
#Threee modules are defined as *helper, multihead attention * input modules

# Helper Modules
class SublayerLogic(nn.Module):

    def __init__(self, model_dimension, dropout_probability):
        super().__init__()
        self.norm = nn.LayerNorm(model_dimension)
        self.dropout = nn.Dropout(p=dropout_probability)

    def forward(self, representations_batch, sublayer_module):
        return representations_batch + self.dropout(sublayer_module(self.norm(representations_batch)))

class DecoderGenerator(nn.Module):

    def __init__(self, model_dimension, vocab_size):
        super().__init__()
        self.linear = nn.Linear(model_dimension, vocab_size)

    def forward(self, trg_representations_batch):
        return torch.sigmoid(self.linear(trg_representations_batch))

class PositionwiseFeedForwardNet(nn.Module):

    def __init__(self, model_dimension, dropout_probability, width_mult=4):
        super().__init__()
        self.linear1 = nn.Linear(model_dimension, width_mult * model_dimension)
        self.linear2 = nn.Linear(width_mult * model_dimension, model_dimension)

        self.dropout = nn.Dropout(p=dropout_probability)
        self.relu = nn.ReLU()

    def forward(self, representations_batch):
        return self.linear2(self.dropout(self.relu(self.linear1(representations_batch))))
     

In [10]:
# Multihead Attention
class MultiHeadedAttention(nn.Module):

    def __init__(self, model_dimension, number_of_heads, dropout_probability, log_attention_weights):
        super().__init__()
        assert model_dimension % number_of_heads == 0, f'Model dimension must be divisible by the number of heads.'

        self.head_dimension = int(model_dimension / number_of_heads)
        self.number_of_heads = number_of_heads

        self.qkv_nets = get_clones(nn.Linear(model_dimension, model_dimension), 3)
        self.out_projection_net = nn.Linear(model_dimension, model_dimension)

        self.attention_dropout = nn.Dropout(p=dropout_probability)
        self.softmax = nn.Softmax(dim=-1)

        self.log_attention_weights = log_attention_weights
        self.attention_weights = None

    def attention(self, query, key, value, mask):
        scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(self.head_dimension)
        if mask is not None:
            scores.masked_fill_(mask == torch.tensor(False), float("-inf"))

        attention_weights = self.softmax(scores)
        attention_weights = self.attention_dropout(attention_weights)

        intermediate_token_representations = torch.matmul(attention_weights, value)

        return intermediate_token_representations, attention_weights

    def forward(self, query, key, value, mask):

        batch_size = query.shape[0]
        query, key, value = [net(x).view(batch_size, -1, self.number_of_heads, self.head_dimension).transpose(1, 2)
                             for net, x in zip(self.qkv_nets, (query, key, value))]

        intermediate_token_representations, attention_weights = self.attention(query, key, value, mask)

        if self.log_attention_weights:
            self.attention_weights = attention_weights

        reshaped = intermediate_token_representations.transpose(1, 2).reshape(batch_size, -1, self.number_of_heads * self.head_dimension)

        token_representations = self.out_projection_net(reshaped)

        return token_representations

In [11]:
# Input Modules
class Embedding(nn.Module):

    def __init__(self, vocab_size, model_dimension):
        super().__init__()
        self.embeddings_table = nn.Linear(vocab_size, model_dimension)
        self.model_dimension = model_dimension

    def forward(self, token_ids_batch):
        assert token_ids_batch.ndim == 3, f'Expected: (batch size, max token sequence length), got {token_ids_batch.shape}'
        embeddings = self.embeddings_table(token_ids_batch)
        return embeddings * math.sqrt(self.model_dimension)

class PositionalEncoding(nn.Module):

    def __init__(self, model_dimension, dropout_probability, expected_max_sequence_length):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout_probability)

        position_id = torch.arange(0, expected_max_sequence_length).unsqueeze(1)
        frequencies = torch.pow(10000., -torch.arange(0, model_dimension, 2, dtype=torch.float) / model_dimension)

        positional_encodings_table = torch.zeros(expected_max_sequence_length, model_dimension)
        positional_encodings_table[:, 0::2] = torch.sin(position_id * frequencies)
        positional_encodings_table[:, 1::2] = torch.cos(position_id * frequencies)

        self.register_buffer('positional_encodings_table', positional_encodings_table)
        self.w2 = nn.Linear(model_dimension * 2, model_dimension)

    def forward(self, embeddings_batch):
        assert embeddings_batch.ndim == 3 and embeddings_batch.shape[-1] == self.positional_encodings_table.shape[1], \
            f'Expected (batch size, max token sequence length, model dimension) got {embeddings_batch.shape}'

        positional_encodings = self.positional_encodings_table[:embeddings_batch.shape[1]]
        size_embeddings_batch = embeddings_batch.shape[0]
        positional_encodings = positional_encodings.unsqueeze(0).expand(size_embeddings_batch, -1 , -1)
        output = self.w2(torch.cat((embeddings_batch, positional_encodings), dim=2))
        return self.dropout(output)

def get_clones(module, num_of_deep_copies):
    return nn.ModuleList([copy.deepcopy(module) for _ in range(num_of_deep_copies)])

In [12]:


# Transformer , transformer neural network is a novel architecture that aims to solve sequence-to-sequence, like traffic count respective to time, day, date, intervals,  while handling long-range dependencies with ease
#Takes (self, model_dimension, input_length, src_vocab_size, trg_vocab_size, number_of_heads, number_of_layers, dropout_probability, log_attention_weights=False)

class Transformer(nn.Module):

    def __init__(self, model_dimension, input_length, src_vocab_size, trg_vocab_size, number_of_heads, number_of_layers, dropout_probability, log_attention_weights=False):
        super().__init__()
        self.src_embedding = Embedding(src_vocab_size, model_dimension)
        self.trg_embedding = Embedding(trg_vocab_size, model_dimension)
        
        self.src_pos_embedding = PositionalEncoding(model_dimension, dropout_probability, input_length)
        self.trg_pos_embedding = PositionalEncoding(model_dimension, dropout_probability, input_length)

        mha = MultiHeadedAttention(model_dimension, number_of_heads, dropout_probability, log_attention_weights)
        pwn = PositionwiseFeedForwardNet(model_dimension, dropout_probability)
        encoder_layer = EncoderLayer(model_dimension, dropout_probability, mha, pwn)
        decoder_layer = DecoderLayer(model_dimension, dropout_probability, mha, pwn)

        self.encoder = Encoder(encoder_layer, number_of_layers)
        self.decoder = Decoder(decoder_layer, number_of_layers)

        self.decoder_generator = DecoderGenerator(model_dimension, trg_vocab_size)
        self.init_params()

    def init_params(self, default_initialization=False):
        if not default_initialization:
            for name, p in self.named_parameters():
                if p.dim() > 1:
                    nn.init.xavier_uniform_(p)

    def forward(self, src_token_ids_batch, trg_token_ids_batch, src_mask, trg_mask, num_pred):
        batch_size = src_token_ids_batch.shape[0]
        num_ahead = num_pred
        src_representations_batch = self.encode(src_token_ids_batch, src_mask)
        trg_log_probs = self.greedy_decoding(src_representations_batch, trg_token_ids_batch, src_mask, batch_size, num_ahead)
        return trg_log_probs

    def encode(self, src_token_ids_batch, src_mask):       
        src_embeddings_batch = self.src_embedding(src_token_ids_batch)
        src_embeddings_batch = self.src_pos_embedding(src_embeddings_batch)
        src_representations_batch = self.encoder(src_embeddings_batch, src_mask)

        return src_representations_batch

    def decode(self, trg_token_ids_batch, src_representations_batch, trg_mask, src_mask):
        trg_embeddings_batch = self.trg_embedding(trg_token_ids_batch)
        trg_embeddings_batch = self.trg_pos_embedding(trg_embeddings_batch)
        trg_representations_batch = self.decoder(trg_embeddings_batch, src_representations_batch, trg_mask, src_mask)
        trg_log_probs = self.decoder_generator(trg_representations_batch)
        return trg_log_probs
    
    def greedy_decoding(self, src_representations_batch, inputs_decode, src_mask, batch_size, num_ahead):
        
        trg_token_ids_batch = inputs_decode[:, -1, :].unsqueeze(1)
        for i in range(num_ahead):
            trg_mask = get_trg_mask(batch_size, i+1).cuda()
            predicted_log_distributions = self.decode(trg_token_ids_batch, src_representations_batch, trg_mask, src_mask)[:,-1:,]
            trg_token_ids_batch = torch.cat((trg_token_ids_batch, predicted_log_distributions), 1)

        output = trg_token_ids_batch[:,1:, :]
        return output

def get_src_mask(batch_size, step_size):
    x = torch.tensor([True])
    src_mask = torch.repeat_interleave(x, step_size*batch_size, dim=0).view(batch_size, 1, 1, -1)
    return src_mask

def get_trg_mask(batch_size, step_size):
    trg_no_look_forward_mask = torch.triu(torch.ones((1, 1, step_size, step_size)) == 1).transpose(2, 3)
    x = torch.tensor([True])
    trg_padding_mask = torch.repeat_interleave(x, step_size*batch_size, dim=0).view(batch_size, 1, 1, -1)
    trg_mask = trg_padding_mask & trg_no_look_forward_mask
    return trg_mask


In [13]:
# Train Process
def TrainModel_Transformer(model, src_mask, trg_mask, train_dataloader, valid_dataloader, num_ahead, num_epochs, learning_rate, patience, min_delta):
    
    loss_MSE = torch.nn.MSELoss()
    optimizer = torch.optim.RMSprop(model.parameters(), lr = learning_rate)

    use_gpu = torch.cuda.is_available()
    
    cur_time = time.time()
    pre_time = time.time()

    # Variables for Early Stopping
    is_best_model = 0
    patient_epoch = 0
       
    for epoch in range(num_epochs):

        total_train_loss = 0
        total_valid_loss = 0

        for data in train_dataloader:

            inputs, labels = data
            batch_size = inputs.shape[0]
            
            if inputs.shape[0] != batch_size:
                continue
                
            if use_gpu:
                inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)
            
            outputs = model(inputs, inputs, src_mask, trg_mask, num_ahead)
            loss_train = loss_MSE(outputs, torch.squeeze(labels))
            total_train_loss += loss_train.data

            optimizer.zero_grad()
            loss_train.backward()
            optimizer.step()
        
        model.eval()

        for data in valid_dataloader:

            inputs, labels = data
            batch_size = inputs.shape[0]
            
            if inputs.shape[0] != batch_size:
                continue
                
            if use_gpu:
                inputs_val, labels_val = Variable(inputs.cuda()), Variable(labels.cuda())
            else:
                inputs_val, labels_val = Variable(inputs), Variable(labels)

            outputs_val = model(inputs_val, inputs_val, src_mask, trg_mask, num_ahead)
            loss_valid = loss_MSE(outputs_val, torch.squeeze(labels_val))
            total_valid_loss += loss_valid.data

        avg_losses_epoch_train = total_train_loss / float(len(train_dataloader))
        avg_losses_epoch_valid = total_valid_loss / float(len(valid_dataloader))

        # Early Stopping
        if epoch == 0:
            is_best_model = 1
            best_model = model
            min_loss_epoch_valid = 10000.0
            if avg_losses_epoch_valid < min_loss_epoch_valid:
                min_loss_epoch_valid = avg_losses_epoch_valid
        else:
            if min_loss_epoch_valid - avg_losses_epoch_valid > min_delta:
                is_best_model = 1
                best_model = model
                min_loss_epoch_valid = avg_losses_epoch_valid
                patient_epoch = 0
            else:
                is_best_model = 0
                patient_epoch += 1
                if patient_epoch >= patience:
                    print('Early Stopped at Epoch:', epoch)
                    break


        cur_time = time.time()

        print('Epoch: {}, train_loss: {}, valid_loss: {}, time: {}, best model: {}'.format( \
                    epoch, \
                    np.around(avg_losses_epoch_train.cpu(), decimals=8),\
                    np.around(avg_losses_epoch_valid.cpu(), decimals=8),\
                    np.around([cur_time - pre_time] , decimals=2),\
                    is_best_model))
        pre_time = cur_time

    return best_model
     

In [14]:
NUM_COL = 87
BATCH_SIZE = 40
INPUT_LEN = 36
PRED_LEN = 12
LEARNING_RATE = 1e-5
NUM_EPOCHS = 100
MIN_DELTA = 5e-4
PATIENCE = 10

BASELINE_MODEL_DIMENSION = 512
BASELINE_MODEL_NUMBER_OF_HEADS = 8
BASELINE_MODEL_NUMBER_OF_LAYERS = 6
BASELINE_MODEL_DROPOUT_PROB = 0.3

In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_gpu = torch.cuda.is_available()
print (use_gpu)

True


In [16]:
!ls

Geoshapefile	  test_data   traffic_forecasting_transformer_11.json
Geoshapefile.zip  test.zip    Traffic_Forecast_TRBAI.ipynb
__MACOSX	  tps_df.pkl  train.zip


In [17]:
data_matrix = load_data("./tps_df.pkl", "2020-01-01 00:00:00.000", "2020-05-31 23:45:00.000", freq="15min")
input_dim = data_matrix.shape[-1] #Input Dimention
train_dataloader, valid_dataloader, _ = prepare_dataloader(data_matrix, input_dim, BATCH_SIZE=BATCH_SIZE, seq_len=INPUT_LEN, pred_len=PRED_LEN, device=device)

src_mask = get_src_mask(BATCH_SIZE, INPUT_LEN).to(device)
trg_mask = get_trg_mask(BATCH_SIZE, PRED_LEN).to(device)

transformer = Transformer(
    model_dimension=BASELINE_MODEL_DIMENSION, 
    input_length=INPUT_LEN, 
    src_vocab_size=NUM_COL, 
    trg_vocab_size=NUM_COL, 
    number_of_heads=BASELINE_MODEL_NUMBER_OF_HEADS, 
    number_of_layers=BASELINE_MODEL_NUMBER_OF_LAYERS, 
    dropout_probability=BASELINE_MODEL_DROPOUT_PROB
).to(device)

transformer_best_model = TrainModel_Transformer(transformer, src_mask, trg_mask, train_dataloader, valid_dataloader, num_ahead=PRED_LEN, learning_rate=LEARNING_RATE, min_delta=MIN_DELTA, num_epochs=NUM_EPOCHS, patience=PATIENCE)

     

Epoch: 0, train_loss: 0.0187818706035614, valid_loss: 0.0016524200327694416, time: [120.27], best model: 1
Epoch: 1, train_loss: 0.009764679707586765, valid_loss: 0.001261229976080358, time: [108.91], best model: 0
Epoch: 2, train_loss: 0.005884480196982622, valid_loss: 0.0008743900107219815, time: [112.18], best model: 1
Epoch: 3, train_loss: 0.0050768498331308365, valid_loss: 0.0007286500185728073, time: [109.24], best model: 0
Epoch: 4, train_loss: 0.004770639818161726, valid_loss: 0.0012073899852111936, time: [108.53], best model: 0
Epoch: 5, train_loss: 0.0044816602021455765, valid_loss: 0.0007421399932354689, time: [107.97], best model: 0
Epoch: 6, train_loss: 0.0042703901417553425, valid_loss: 0.0012284900294616818, time: [108.69], best model: 0
Epoch: 7, train_loss: 0.004115779884159565, valid_loss: 0.0006634300225414336, time: [108.13], best model: 0
Epoch: 8, train_loss: 0.003937370143830776, valid_loss: 0.0008675199933350086, time: [108.19], best model: 0
Epoch: 9, train_los

In [18]:
%cd '/content/drive/MyDrive/TRAFFIC4CAST_TRBAI'
import os 


/content/drive/MyDrive/TRAFFIC4CAST_TRBAI


In [19]:
'''
Step1: read and reshape data
Step2: feed into trained model
Step3: convert output to JSON file
Here we take one testing data as an example to convert it to JSON file.
In this challenge, you should convert all 15 predicting results to one JSON file.
Please check https://colab.research.google.com/drive/1Hkt3kQuh7WzwUTnLgKCcvfPAV1CUK8lF?usp=sharing for more information of the expected result.
'''

test_files = os.listdir('test_data')




In [21]:
test_files.remove('tps_tot')

In [22]:
test_files

['tps_1.pkl',
 'tps_2.pkl',
 'tps_3.pkl',
 'tps_4.pkl',
 'tps_5.pkl',
 'tps_6.pkl',
 'tps_7.pkl',
 'tps_8.pkl',
 'tps_9.pkl',
 'tps_10.pkl',
 'tps_11.pkl',
 'tps_12.pkl',
 'tps_13.pkl',
 'tps_14.pkl',
 'tps_15.pkl']

In [23]:
out_cols = ['11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22',
       '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34',
       '35', '36', '37', '38', '39', '40', '42', '43', '44', '45', '46', '47',
       '49', '50', '51', '52', '53', '54', '55', '58', '59', '60', '61', '62',
       '63', '64', '65', '66', '67', '68', '69', '70', '73', '74', '75', '76',
       '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88',
       '89', '90', '91', '92', '93', '94', '95', '96', '97', '99', '100',
       '102', '103', '104', '106']

horizon = {'tps_1.pkl':['2020-06-02 06:15:00', '2020-06-02 09:00:00'],
           'tps_2.pkl':['2020-06-03 07:15:00', '2020-06-03 10:00:00'],
           'tps_3.pkl':['2020-06-04 08:15:00', '2020-06-04 11:00:00'],
           'tps_4.pkl':['2020-06-05 09:15:00', '2020-06-05 12:00:00'],
           'tps_5.pkl':['2020-06-06 10:15:00', '2020-06-06 13:00:00'],
           'tps_6.pkl':['2020-06-07 11:15:00', '2020-06-07 14:00:00'],
           'tps_7.pkl':['2020-06-08 12:15:00', '2020-06-08 15:00:00'],
           'tps_8.pkl':['2020-06-09 13:15:00', '2020-06-09 16:00:00'],
           'tps_9.pkl':['2020-06-10 14:15:00', '2020-06-10 17:00:00'],
           'tps_10.pkl':['2020-06-11 15:15:00', '2020-06-11 18:00:00'],
           'tps_11.pkl':['2020-06-12 16:15:00', '2020-06-12 19:00:00'],
           'tps_12.pkl':['2020-06-13 17:15:00', '2020-06-13 20:00:00'],
           'tps_13.pkl':['2020-06-14 18:15:00', '2020-06-14 21:00:00'],
           'tps_14.pkl':['2020-06-15 19:15:00', '2020-06-15 22:00:00'],
           'tps_15.pkl':['2020-06-16 20:15:00', '2020-06-16 23:00:00'],
           }
out_df = pd.DataFrame(columns = out_cols)

In [24]:
BATCH_SIZE_TEST = 1
for test_file in test_files:
  print (os.path.join('test_data',test_file))
  tps__raw = pd.read_pickle(os.path.join('test_data',test_file))      #Read each test data in the loop
  reshaped_tps_df = reshape_data(tps__raw)                            #Reshape
  reshaped_tps_df = reshaped_tps_df.set_index('TIME')                 #final format
  reshaped_tps_value = reshaped_tps_df.values
  reshaped_tps_value = np.expand_dims(reshaped_tps_value, axis=0)
  reshaped_tps_value = torch.from_numpy(reshaped_tps_value).float().to(device)    #Tensors

  src_mask_test = get_src_mask(BATCH_SIZE_TEST, INPUT_LEN).to(device)
  trg_mask_test = get_trg_mask(BATCH_SIZE_TEST, PRED_LEN).to(device)


  transformer_out = transformer_best_model(reshaped_tps_value, reshaped_tps_value, src_mask_test, trg_mask_test, PRED_LEN).squeeze(0)       #Model Test
  output_1 = pd.DataFrame(transformer_out.cpu().detach().numpy())
  st = horizon[test_file][0]; et = horizon[test_file][1]
  print (st, et)
  output_1.index = pd.date_range(start=st, end=et, freq='15min').astype(int) / 10**9

  # output_1.index = pd.date_range(start='2020-06-02 06:15:00', end='2020-06-02 09:00:00', freq='15min').astype(int) / 10**9
  output_1.columns = reshaped_tps_df.columns

  # result = pd.concat([output_1, output_2, ..., output_15 ])
  # result = pd.concat([output_1])
  out_df = pd.concat([out_df,output_1])
out_df.to_json('./traffic_forecasting_transformer_Final__.json')

test_data/tps_1.pkl
2020-06-02 06:15:00 2020-06-02 09:00:00
test_data/tps_2.pkl


<ipython-input-24-0a4459775fa2>:19: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  output_1.index = pd.date_range(start=st, end=et, freq='15min').astype(int) / 10**9


2020-06-03 07:15:00 2020-06-03 10:00:00
test_data/tps_3.pkl


<ipython-input-24-0a4459775fa2>:19: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  output_1.index = pd.date_range(start=st, end=et, freq='15min').astype(int) / 10**9


2020-06-04 08:15:00 2020-06-04 11:00:00
test_data/tps_4.pkl


...

<ipython-input-24-0a4459775fa2>:19: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  output_1.index = pd.date_range(start=st, end=et, freq='15min').astype(int) / 10**9


2020-06-15 19:15:00 2020-06-15 22:00:00
test_data/tps_15.pkl


<ipython-input-24-0a4459775fa2>:19: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  output_1.index = pd.date_range(start=st, end=et, freq='15min').astype(int) / 10**9


2020-06-16 20:15:00 2020-06-16 23:00:00


<ipython-input-24-0a4459775fa2>:19: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  output_1.index = pd.date_range(start=st, end=et, freq='15min').astype(int) / 10**9


In [119]:
results=out_df

In [82]:
type(r.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [83]:
type(t.index)

pandas.core.indexes.datetimes.DatetimeIndex

In [86]:
type(results)

pandas.core.frame.DataFrame

In [106]:
results

,11,12,13,14,15,16,17,18,19,20,...,94,95,96,97,99,100,102,103,104,106
1.591078e+09,0.889416,0.942744,0.803007,0.826039,0.416141,0.592024,0.589802,0.941366,0.978600,0.945156,...,0.986868,0.913124,0.984713,0.997592,0.995314,0.982070,0.996875,0.994130,0.997549,0.997496
1.591079e+09,0.871052,0.926781,0.747571,0.724171,0.347129,0.526694,0.541750,0.927454,0.974638,0.942935,...,0.982813,0.896758,0.978772,0.996863,0.992823,0.976419,0.996784,0.990839,0.996987,0.996792
1.591080e+09,0.863541,0.913180,0.726496,0.668915,0.308150,0.509234,0.535276,0.918801,0.972802,0.943827,...,0.979031,0.895604,0.973268,0.996278,0.990886,0.971372,0.996826,0.988998,0.996646,0.996512
1.591081e+09,0.856056,0.908220,0.711505,0.634797,0.273876,0.493061,0.534181,0.907486,0.970534,0.942829,...,0.975911,0.892745,0.967883,0.995669,0.989201,0.964346,0.996773,0.987929,0.996374,0.996376
1.591082e+09,0.840807,0.907768,0.690233,0.597872,0.230306,0.467433,0.527047,0.891720,0.966493,0.939211,...,0.970929,0.883418,0.959568,0.994482,0.986667,0.951894,0.996467,0.987088,0.995886,0.996267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.592345e+09,0.989025,0.992853,0.972720,0.978872,0.963002,0.952459,0.966103,0.981421,0.988917,0.988137,...,0.987508,0.983524,0.992963,0.998287,0.995142,0.993123,0.997663,0.995354,0.998046,0.996605
1.592346e+09,0.989225,0.993043,0.973343,0.979221,0.963635,0.953406,0.967059,0.981692,0.989188,0.988449,...,0.987450,0.983677,0.993193,0.998245,0.995010,0.992858,0.997661,0.995223,0.997975,0.996622
1.592347e+09,0.989390,0.993194,0.973926,0.979783,0.964253,0.954402,0.968106,0.981929,0.989495,0.988600,...,0.987636,0.983836,0.993506,0.998293,0.995059,0.992758,0.997617,0.995170,0.997977,0.996635
1.592348e+09,0.989395,0.993223,0.974082,0.980174,0.964429,0.954591,0.968465,0.982101,0.989704,0.988553,...,0.987878,0.983947,0.993761,0.998371,0.995152,0.992686,0.997534,0.995206,0.998024,0.996628


In [121]:
results=results*1000
results=results.astype(int)

In [115]:
results=results/1000
results.to_json('Transformer_TRAFIC4Cast_Final_Results.json')

In [124]:
final=results/1000


,11,12,13,14,15,16,17,18,19,20,...,94,95,96,97,99,100,102,103,104,106
1.591078e+09,0.889,0.942,0.803,0.826,0.416,0.592,0.589,0.941,0.978,0.945,...,0.986,0.913,0.984,0.997,0.995,0.982,0.996,0.994,0.997,0.997
1.591079e+09,0.871,0.926,0.747,0.724,0.347,0.526,0.541,0.927,0.974,0.942,...,0.982,0.896,0.978,0.996,0.992,0.976,0.996,0.990,0.996,0.996
1.591080e+09,0.863,0.913,0.726,0.668,0.308,0.509,0.535,0.918,0.972,0.943,...,0.979,0.895,0.973,0.996,0.990,0.971,0.996,0.988,0.996,0.996
1.591081e+09,0.856,0.908,0.711,0.634,0.273,0.493,0.534,0.907,0.970,0.942,...,0.975,0.892,0.967,0.995,0.989,0.964,0.996,0.987,0.996,0.996
1.591082e+09,0.840,0.907,0.690,0.597,0.230,0.467,0.527,0.891,0.966,0.939,...,0.970,0.883,0.959,0.994,0.986,0.951,0.996,0.987,0.995,0.996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.592345e+09,0.989,0.992,0.972,0.978,0.963,0.952,0.966,0.981,0.988,0.988,...,0.987,0.983,0.992,0.998,0.995,0.993,0.997,0.995,0.998,0.996
1.592346e+09,0.989,0.993,0.973,0.979,0.963,0.953,0.967,0.981,0.989,0.988,...,0.987,0.983,0.993,0.998,0.995,0.992,0.997,0.995,0.997,0.996
1.592347e+09,0.989,0.993,0.973,0.979,0.964,0.954,0.968,0.981,0.989,0.988,...,0.987,0.983,0.993,0.998,0.995,0.992,0.997,0.995,0.997,0.996
1.592348e+09,0.989,0.993,0.974,0.980,0.964,0.954,0.968,0.982,0.989,0.988,...,0.987,0.983,0.993,0.998,0.995,0.992,0.997,0.995,0.998,0.996


In [125]:
final.to_json('Transformet_Traffic4Cast_Final_Results.json')

In [92]:
results.to_json('./__Transformer_traffic_forecasting_transformer_Final__.json')

In [93]:
r1=results

In [95]:
r1.to_json('pred.json')

In [96]:
pd.read_json('pred.json')

,11,12,13,14,15,16,17,18,19,20,...,94,95,96,97,99,100,102,103,104,106
2020-06-02 06:15:00,0.889,0.943,0.803,0.826,0.416,0.592,0.590,0.941,0.979,0.945,...,0.987,0.913,0.985,0.998,0.995,0.982,0.997,0.994,0.998,0.997
2020-06-02 06:30:00,0.871,0.927,0.748,0.724,0.347,0.527,0.542,0.927,0.975,0.943,...,0.983,0.897,0.979,0.997,0.993,0.976,0.997,0.991,0.997,0.997
2020-06-02 06:45:00,0.864,0.913,0.726,0.669,0.308,0.509,0.535,0.919,0.973,0.944,...,0.979,0.896,0.973,0.996,0.991,0.971,0.997,0.989,0.997,0.997
2020-06-02 07:00:00,0.856,0.908,0.712,0.635,0.274,0.493,0.534,0.907,0.971,0.943,...,0.976,0.893,0.968,0.996,0.989,0.964,0.997,0.988,0.996,0.996
2020-06-02 07:15:00,0.841,0.908,0.690,0.598,0.230,0.467,0.527,0.892,0.966,0.939,...,0.971,0.883,0.960,0.994,0.987,0.952,0.996,0.987,0.996,0.996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-16 22:00:00,0.989,0.993,0.973,0.979,0.963,0.952,0.966,0.981,0.989,0.988,...,0.988,0.984,0.993,0.998,0.995,0.993,0.998,0.995,0.998,0.997
2020-06-16 22:15:00,0.989,0.993,0.973,0.979,0.964,0.953,0.967,0.982,0.989,0.988,...,0.987,0.984,0.993,0.998,0.995,0.993,0.998,0.995,0.998,0.997
2020-06-16 22:30:00,0.989,0.993,0.974,0.980,0.964,0.954,0.968,0.982,0.989,0.989,...,0.988,0.984,0.994,0.998,0.995,0.993,0.998,0.995,0.998,0.997
2020-06-16 22:45:00,0.989,0.993,0.974,0.980,0.964,0.955,0.968,0.982,0.990,0.989,...,0.988,0.984,0.994,0.998,0.995,0.993,0.998,0.995,0.998,0.997


In [99]:
format=pd.read_json('predict_result_format.json')

In [100]:
format

,11,12,13,14,15,16,17,18,19,20,...,94,95,96,97,99,100,102,103,104,106
2020-06-02 06:15:00,0.061,0.884,0.370,0.568,0.823,0.862,0.503,0.629,0.380,0.321,...,0.569,0.095,0.487,0.598,0.272,0.859,0.958,0.233,0.994,0.149
2020-06-02 06:30:00,0.789,0.272,0.760,0.815,0.826,0.330,0.804,0.149,0.544,0.752,...,0.896,0.020,0.848,0.794,0.430,0.490,0.136,0.846,0.773,0.134
2020-06-02 06:45:00,0.929,0.210,0.615,0.545,0.062,0.456,0.689,0.381,0.460,0.863,...,0.574,0.038,0.807,0.510,0.425,0.852,0.057,0.688,0.156,0.031
2020-06-02 07:00:00,0.282,0.023,0.326,0.290,0.882,0.764,0.586,0.566,0.467,0.972,...,0.107,0.675,0.007,0.106,0.201,0.464,0.260,0.228,0.510,0.367
2020-06-02 07:15:00,0.069,0.040,0.469,0.992,0.698,0.472,0.424,0.273,0.221,0.665,...,0.783,0.224,0.073,0.569,0.829,0.128,0.148,0.444,0.066,0.261
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-06-16 22:00:00,0.639,0.248,0.062,0.774,0.496,0.455,0.150,0.754,0.386,0.222,...,0.575,0.973,0.013,0.151,0.748,0.880,0.919,0.126,0.332,0.060
2020-06-16 22:15:00,0.233,0.902,0.367,0.947,0.405,0.484,0.428,0.952,0.133,0.942,...,0.857,0.658,0.282,0.198,0.193,0.928,0.244,0.575,0.914,0.046
2020-06-16 22:30:00,0.373,0.001,0.003,0.050,0.853,0.366,0.307,0.588,0.451,0.158,...,0.124,0.216,0.169,0.516,0.010,0.944,0.808,0.026,0.961,0.212
2020-06-16 22:45:00,0.919,0.406,0.611,0.573,0.453,0.667,0.040,0.772,0.073,0.984,...,0.435,0.594,0.523,0.936,0.107,0.115,0.880,0.486,0.179,0.758


In [101]:
format=results
format

,11,12,13,14,15,16,17,18,19,20,...,94,95,96,97,99,100,102,103,104,106
1.591078e+09,0.889,0.943,0.803,0.826,0.416,0.592,0.590,0.941,0.979,0.945,...,0.987,0.913,0.985,0.998,0.995,0.982,0.997,0.994,0.998,0.997
1.591079e+09,0.871,0.927,0.748,0.724,0.347,0.527,0.542,0.927,0.975,0.943,...,0.983,0.897,0.979,0.997,0.993,0.976,0.997,0.991,0.997,0.997
1.591080e+09,0.864,0.913,0.726,0.669,0.308,0.509,0.535,0.919,0.973,0.944,...,0.979,0.896,0.973,0.996,0.991,0.971,0.997,0.989,0.997,0.997
1.591081e+09,0.856,0.908,0.712,0.635,0.274,0.493,0.534,0.907,0.971,0.943,...,0.976,0.893,0.968,0.996,0.989,0.964,0.997,0.988,0.996,0.996
1.591082e+09,0.841,0.908,0.690,0.598,0.230,0.467,0.527,0.892,0.966,0.939,...,0.971,0.883,0.960,0.994,0.987,0.952,0.996,0.987,0.996,0.996
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1.592345e+09,0.989,0.993,0.973,0.979,0.963,0.952,0.966,0.981,0.989,0.988,...,0.988,0.984,0.993,0.998,0.995,0.993,0.998,0.995,0.998,0.997
1.592346e+09,0.989,0.993,0.973,0.979,0.964,0.953,0.967,0.982,0.989,0.988,...,0.987,0.984,0.993,0.998,0.995,0.993,0.998,0.995,0.998,0.997
1.592347e+09,0.989,0.993,0.974,0.980,0.964,0.954,0.968,0.982,0.989,0.989,...,0.988,0.984,0.994,0.998,0.995,0.993,0.998,0.995,0.998,0.997
1.592348e+09,0.989,0.993,0.974,0.980,0.964,0.955,0.968,0.982,0.990,0.989,...,0.988,0.984,0.994,0.998,0.995,0.993,0.998,0.995,0.998,0.997


In [102]:
format.to_json("prd.json")

In [ ]:
BATCH_SIZE_TEST = 1
reshaped_tps_df = reshape_data(tps_mrgd)
reshaped_tps_df = reshaped_tps_df.set_index('TIME')
reshaped_tps_value = reshaped_tps_df.values
reshaped_tps_value = np.expand_dims(reshaped_tps_value, axis=0)
reshaped_tps_value = torch.from_numpy(reshaped_tps_value).float().to(device)

src_mask_test = get_src_mask(BATCH_SIZE_TEST, INPUT_LEN).to(device)
trg_mask_test = get_trg_mask(BATCH_SIZE_TEST, PRED_LEN).to(device)

In [ ]:
#Test Data set is reshaped and prepaed to feed into the model
transformer_out = transformer_best_model(reshaped_tps_value, reshaped_tps_value, src_mask_test, trg_mask_test, PRED_LEN).squeeze(0)


In [ ]:
#Output 1, test data 1
output_1 = pd.DataFrame(transformer_out.cpu().detach().numpy())
st = horizon['tps_1.pkl'][0]; et = horizon['tps_1.pkl'][1]
print (st, et)
output_1.index = pd.date_range(start=st, end=et, freq='15min').astype(int)

2020-06-02 06:15:00 2020-06-02 09:00:00


<ipython-input-46-d638e6c43f60>:5: FutureWarning: casting datetime64[ns] values to int64 with .astype(...) is deprecated and will raise in a future version. Use .view(...) instead.
  output_1.index = pd.date_range(start=st, end=et, freq='15min').astype(int)


In [ ]:
# output_1.index = pd.date_range(start='2020-06-02 06:15:00', end='2020-06-02 09:00:00', freq='15min').astype(int) / 10**9
output_1.columns = reshaped_tps_df.columns


In [ ]:
# result = pd.concat([output_1, output_2, ..., output_15 ])
  # result = pd.concat([output_1])
out_df11 = pd.concat([out_df,output_1])

In [ ]:
out_df11.to_json('./traffic_forecasting_transformer_11.json')

In [ ]:
BATCH_SIZE_TEST = 1
reshaped_tps_df = reshape_data(tps_mrgd)
reshaped_tps_df = reshaped_tps_df.set_index('TIME')
reshaped_tps_value = reshaped_tps_df.values
reshaped_tps_value = np.expand_dims(reshaped_tps_value, axis=0)
reshaped_tps_value = torch.from_numpy(reshaped_tps_value).float().to(device)

src_mask_test = get_src_mask(BATCH_SIZE_TEST, INPUT_LEN).to(device)
trg_mask_test = get_trg_mask(BATCH_SIZE_TEST, PRED_LEN).to(device)

In [ ]:
#Test Data set total is already reshaped and prepaed to feed into the model
transformer_out = transformer_best_model(reshaped_tps_value, reshaped_tps_value, src_mask_test, trg_mask_test, PRED_LEN).squeeze(0)

RuntimeError: ignored

In [ ]:
horizon

{'tps_1.pkl': ['2020-06-02 06:15:00', '2020-06-02 09:00:00'],
 'tps_2.pkl': ['2020-06-03 07:15:00', '2020-06-03 10:00:00'],
 'tps_3.pkl': ['2020-06-04 08:15:00', '2020-06-04 11:00:00'],
 'tps_4.pkl': ['2020-06-05 09:15:00', '2020-06-05 12:00:00'],
 'tps_5.pkl': ['2020-06-06 10:15:00', '2020-06-06 13:00:00'],
 'tps_6.pkl': ['2020-06-07 11:15:00', '2020-06-07 14:00:00'],
 'tps_7.pkl': ['2020-06-08 12:15:00', '2020-06-08 15:00:00'],
 'tps_8.pkl': ['2020-06-09 13:15:00', '2020-06-09 16:00:00'],
 'tps_9.pkl': ['2020-06-10 14:15:00', '2020-06-10 17:00:00'],
 'tps_10.pkl': ['2020-06-11 15:15:00', '2020-06-11 18:00:00'],
 'tps_11.pkl': ['2020-06-12 16:15:00', '2020-06-12 19:00:00'],
 'tps_12.pkl': ['2020-06-13 17:15:00', '2020-06-13 20:00:00'],
 'tps_13.pkl': ['2020-06-14 18:15:00', '2020-06-14 21:00:00'],
 'tps_14.pkl': ['2020-06-15 19:15:00', '2020-06-15 22:00:00'],
 'tps_15.pkl': ['2020-06-16 20:15:00', '2020-06-16 23:00:00']}

In [ ]:
{horizon['tps_1.pkl'][0],horizon['tps_2.pkl'][0]}
st1 = {horizon['tps_1.pkl'][0],horizon['tps_2.pkl'][0],horizon['tps_3.pkl'][0],horizon['tps_4.pkl'][0],horizon['tps_5.pkl'][0],horizon['tps_6.pkl'][0],horizon['tps_7.pkl'][0],
      horizon['tps_8.pkl'][0],horizon['tps_9.pkl'][0],horizon['tps_10.pkl'][0],horizon['tps_11.pkl'][0],horizon['tps_12.pkl'][0],horizon['tps_13.pkl'][0],
      horizon['tps_14.pkl'][0],horizon['tps_15.pkl'][0]}
st1

{'2020-06-02 06:15:00',
 '2020-06-03 07:15:00',
 '2020-06-04 08:15:00',
 '2020-06-05 09:15:00',
 '2020-06-06 10:15:00',
 '2020-06-07 11:15:00',
 '2020-06-08 12:15:00',
 '2020-06-09 13:15:00',
 '2020-06-10 14:15:00',
 '2020-06-11 15:15:00',
 '2020-06-12 16:15:00',
 '2020-06-13 17:15:00',
 '2020-06-14 18:15:00',
 '2020-06-15 19:15:00',
 '2020-06-16 20:15:00'}

In [ ]:
#Output total, test data merged
output_tot = pd.DataFrame(transformer_out.cpu().detach().numpy())
st = {horizon['tps_1.pkl'][0],horizon['tps_2.pkl'][0],horizon['tps_3.pkl'][0],horizon['tps_4.pkl'][0],horizon['tps_5.pkl'][0],horizon['tps_6.pkl'][0],horizon['tps_7.pkl'][0],
      horizon['tps_8.pkl'][0],horizon['tps_9.pkl'][0],horizon['tps_10.pkl'][0],horizon['tps_11.pkl'][0],horizon['tps_12.pkl'][0],horizon['tps_13.pkl'][0],
      horizon['tps_14.pkl'][0],horizon['tps_15.pkl'][0]}
et = {horizon['tps_1.pkl'][1],horizon['tps_2.pkl'][1],horizon['tps_3.pkl'][1],horizon['tps_4.pkl'][1],horizon['tps_5.pkl'][1],horizon['tps_6.pkl'][1],horizon['tps_7.pkl'][1],
      horizon['tps_8.pkl'][1],horizon['tps_9.pkl'][1],horizon['tps_10.pkl'][1],horizon['tps_11.pkl'][1],horizon['tps_12.pkl'][1],horizon['tps_13.pkl'][1],
      horizon['tps_14.pkl'][1],horizon['tps_15.pkl'][1]}
print (st, et)
output_1.index = pd.date_range(start=st, end=et, freq='15min').astype(int)

{'2020-06-12 16:15:00', '2020-06-04 08:15:00', '2020-06-07 11:15:00', '2020-06-14 18:15:00', '2020-06-11 15:15:00', '2020-06-06 10:15:00', '2020-06-16 20:15:00', '2020-06-02 06:15:00', '2020-06-03 07:15:00', '2020-06-08 12:15:00', '2020-06-10 14:15:00', '2020-06-15 19:15:00', '2020-06-13 17:15:00', '2020-06-09 13:15:00', '2020-06-05 09:15:00'} {'2020-06-06 13:00:00', '2020-06-05 12:00:00', '2020-06-02 09:00:00', '2020-06-09 16:00:00', '2020-06-10 17:00:00', '2020-06-14 21:00:00', '2020-06-04 11:00:00', '2020-06-16 23:00:00', '2020-06-08 15:00:00', '2020-06-11 18:00:00', '2020-06-12 19:00:00', '2020-06-03 10:00:00', '2020-06-13 20:00:00', '2020-06-07 14:00:00', '2020-06-15 22:00:00'}


TypeError: ignored